In [ ]:
%pylab notebook
from glob import glob
import json
import os.path

π = np.pi

In [ ]:
# List filenames
dr9_runs = glob(os.path.expanduser('~/SIMU/20200124/dynamic_run_9_*.json')) ; dr9_runs.sort()

# Obtain simulation parameters
with open('/home/pierre/code/RMPCDMD/experiments/04-enzyme/pfiles_dynamic_run_9/dynamic_run_9_20200304_00001.json', 'r') as f:
    input_data = json.load(f)                                     


In [ ]:
# Collect data from simulations

all_rates_data = [] 
for fn in dr9_runs: 
    with open(fn, 'r') as f:
        data = json.load(f)
    all_rates_data.append(data)


In [ ]:
# Structured data type for the data
e_sim_dt = np.dtype([
    ('n_e0', float),
    ('phi', float),
    ('n_fluid', float),
    ('n_s', float),
    ('n_p', float),
    ('n_estar', float),
    ('n_e', float),
    ('n_s_plus', float),
    ('n_p_plus', float),
    ('n_s_minus', float),
    ('n_p_minus', float),
])

In [ ]:
# Array for simulation data
simu = np.zeros(4, dtype=e_sim_dt)

In [ ]:
# Read the simulation data

n_solvent = np.array([d['n_solvent']['mean'] for d in all_rates_data])
n_minus = np.array([d['n_minus']['mean'] for d in all_rates_data])
n_plus = np.array([d['n_plus']['mean'] for d in all_rates_data])

ρ = 12                                             
V = 30**3
v_e = 4/3*pi * (3**3 + 2)
substrate_fraction = float(input_data['substrate_fraction'])
ρ = int(input_data['rho'])

for i, d in enumerate(all_rates_data):
    n_e0 = d['N_enzymes']
    ϕ = n_e0 * v_e / V
    n_fluid = ρ * (V - int(n_e0*v_e))
    n_solvent = np.array(d['n_solvent']['mean'])
    n_plus = np.array(d['n_plus']['mean'])
    n_minus = np.array(d['n_minus']['mean'])
    n_estar = (n_fluid*substrate_fraction - n_solvent.sum())
    n_e = n_e0 - n_estar
    simu[i]['n_e0'] = n_e0
    bind_time = (d['bind product time'] + d['bind substrate time'])/2
    release_time = (d['release product time'] + d['release substrate time'])/2
    simu[i]['n_e'] = n_e0 * bind_time/(bind_time+release_time)
    simu[i]['n_estar'] = n_e0 * release_time/(bind_time+release_time)

    simu[i]['phi'] = ϕ
    simu[i]['n_fluid'] = n_fluid
    simu[i]['n_s'] = n_solvent[0]
    simu[i]['n_p'] = n_solvent[1]
    simu[i]['n_s_plus'] = n_plus[0]
    simu[i]['n_p_plus'] = n_plus[1]
    simu[i]['n_s_minus'] = n_minus[0]
    simu[i]['n_p_minus'] = n_minus[1]


In [ ]:
# Compute model estimates

R = float(input_data['sigma_E'])*2**(1/6)    
V = np.product(list(map(float, input_data['L'].split())))

sigma_N = list(map(float, input_data['sigma_N'].split()))

k1_0 = float(input_data['proba_s']) * np.sqrt(8*π*float(input_data['T'])) * R**2
k2_0 = float(input_data['proba_p']) * np.sqrt(8*π*float(input_data['T'])) * R**2
km1 = float(input_data['rate_release_s'])
km2 = float(input_data['rate_release_p'])
kappa = km1+km2

model = np.zeros(4, dtype=e_sim_dt)

for i in range(4):
    n_e0 = simu[i]['n_e0']
    model[i]['n_e0'] = n_e0
    e0 = n_e0 / V
    v_real = V - n_e0 * v_e
    s0 = simu[i]['n_fluid'] *substrate_fraction / V
    phi = simu[i]['phi']

    # Solve

    γ = 1/(1-phi)
    k1 = k1_0*γ
    k2 = k2_0*γ
    μ = k1/(1+k1*km2/km1/k2) + k2/(km1*k2/k1/km2 + 1)
    Δ = (μ*(e0+s0) + kappa)**2 - 4*μ**2*s0*e0

    estar = (( μ*(s0+e0)+kappa) - np.sqrt(Δ)) / (2*μ)
    S = (s0-estar) / (1+k1*km2/km1/k2)
    P = (s0-estar) / (km1*k2/k1/km2 + 1)
    
    #print(estar, simu[i]['n_estar']/V)
    #print(simu[i]['n_s']/V, S)
    model[i]['phi'] = phi
    model[i]['n_s'] = S*V
    model[i]['n_p'] = P*V
    model[i]['n_e'] = (e0 - estar)*V
    model[i]['n_estar'] = estar*V


In [ ]:
simu['n_e0']

In [ ]:
plt.figure()
plt.subplot(211)

plt.plot(simu['phi'], simu['n_s_minus'] / simu['n_e0'], label=r'$S^- / N_E$')
plt.plot(simu['phi'], simu['n_s']*simu['n_e']*k1_0/V/(1-simu['phi']) / simu['n_e0'], label=r'$S\ E\ k_1^0 / N_E$')
plt.plot(simu['phi'], model['n_s']*model['n_e']*k1_0/V/(1-model['phi']) / model['n_e0'], label=r'$S\ E\ k_1^0 / N_E$ (model)')

plt.xlabel(r'$\phi$')
plt.legend()

plt.subplot(212)

plt.plot(simu['phi'], simu['n_p_minus'] / simu['n_e0'], label=r'$P^- / N_E$')
plt.plot(simu['phi'], simu['n_p']*simu['n_e']*k2_0/V/(1-simu['phi']) / simu['n_e0'], label=r'$P\ E\ k_2^0 / N_E$')
plt.plot(simu['phi'], model['n_p']*model['n_e']*k2_0/V/(1-model['phi']) / model['n_e0'], label=r'$P\ E\ k_2^0 / N_E$ (model)')

plt.xlabel(r'$\phi$')
plt.legend()

plt.suptitle('Actual consumption rates and kinetic theory')

In [ ]:
plt.figure()

plt.subplot(211)

plt.plot(simu['phi'], simu['n_s_plus'] / V / simu['n_e0'], label=r'$S^+$')
plt.plot(simu['phi'], simu['n_estar'] / V * km1/ simu['n_e0'], label=r'$E* k^{-1}$')
plt.plot(simu['phi'], model['n_estar'] / V * km1/ model['n_e0'], label=r'$E* k_{-1}$ (model)')


plt.xlabel(r'$\phi$')
plt.legend()

plt.subplot(212)

plt.plot(simu['phi'], simu['n_p_plus'] / V/ simu['n_e0'], label=r'$P^+$')
plt.plot(simu['phi'], simu['n_estar'] / V * km2/ simu['n_e0'], label=r'$E* k_{-2}$')
plt.plot(simu['phi'], model['n_estar'] / V * km2/ model['n_e0'], label=r'$E* k_{-2}$ (model)')

ylo, yhi = plt.ylim()

#plt.ylim(0, yhi)
plt.xlabel(r'$\phi$')
plt.legend()


In [ ]:
plt.figure()
plt.plot(simu['phi'], simu['n_estar'], label='E* simu', marker='o')
plt.plot(simu['phi'], simu['n_e'], label='E simu', marker='^')
#plt.plot(simu['phi'], simu['n_estar'] + simu['n_e'], label='E+E* simu')

plt.plot(simu['phi'], model['n_estar'], label='E* model')
plt.plot(simu['phi'], model['n_e'], label='E model')
#plt.plot(simu['phi'], model['n_estar'] + model['n_e'], label='E+E* model')

#plt.plot(simu['phi'], simu['n_e0'])

plt.xlabel(r'$\phi$')
plt.legend()